In [ ]:
from PIL import Image
import requests
from transformers import SamModel, SamProcessor

model = SamModel.from_pretrained("facebook/sam-vit-base")
processor = SamProcessor.from_pretrained("facebook/sam-vit-base")
model.to("cuda")

raw_image = Image.open("../1_media/input_images/cat.jpg")
input_points = [[[450, 600]]] 


In [ ]:
inputs = processor(raw_image, input_points=input_points, return_tensors="pt").to("cuda")
outputs = model(**inputs)
masks = processor.image_processor.post_process_masks(outputs.pred_masks.cpu(), inputs["original_sizes"].cpu(), inputs["reshaped_input_sizes"].cpu())
scores = outputs.iou_scores

In [ ]:
mask_image = masks[0].squeeze(0)
mask_image = (mask_image * 255).cpu().numpy()
mask_image = mask_image.astype("uint8").transpose(1, 2, 0)
mask_image = Image.fromarray(mask_image)
mask_image.save("cat_mask.jpg")